# Importing Dependencies

In [1]:
import cv2
import numpy as np
from keras.models import load_model
from keras.applications import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os
import glob
import pickle

- PLease note that most **print()** statement in my codes were introduced for debugging purpose, i will remove irrelevant ones when i convinced of proper functionalities

### Face detection Function

In [14]:
# Step 1: Face Detection with HAAR Cascade
def detect_faces(image_path):
    # Load the HAAR cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Read the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    
    
    # Detect faces
    # highest_Neigbor = 0
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # Create a directory for the new image
    working_dir = os.getcwd()
    folder_path =os.path.join(working_dir, 'cropped')
    
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # If faces are detected, crop and save them
    cropped_faces = []
    for (x, y, w, h) in faces:
        cropped_face = image[y:y+h, x:x+w]
        cropped_faces.append(cropped_face)
        # Save the cropped face
        # cv2.imwrite(f"{os.path.basename(image_path)}", cropped_face)
        cv2.imwrite(os.path.join(folder_path, os.path.basename(image_path)), cropped_face)

    
    return cropped_faces




# VGG16 Model

In [21]:
#  Initialize VGG16

# Load the pre-trained VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=False, pooling='avg')  # Use average pooling


### Get Embeddings Function

In [20]:
# Step 2: Generate Embeddings using Pre-trained VGG16 Model
def get_embeddings(face_images, model):
    embeddings = []
    
    for face in face_images:
        # Resize and preprocess the face image
        face_resized = cv2.resize(face, (160, 160))  # Resize for VGG16 input
        face_array = image.img_to_array(face_resized)
        face_array = np.expand_dims(face_array, axis=0)
        face_array = preprocess_input(face_array)
        
        # Generate embedding
        embedding = model.predict(face_array)
        embeddings.append(embedding.flatten())  # Flatten the output to 1D
    
    return embeddings




In [ ]:
def to_npz_file():
    for path in the_cropped_paths:
        img = cv2.imread(path)
        train_images.append(img)
    train_images = np.array(train_images)
    np.savez('cropped_faces', train = train_images)


### SVM training function

In [4]:
# Step 3: Train SVM Classifier
def train_svm(embeddings, labels):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)
    
    # Initialize and train the SVM classifier
    clf = svm.SVC(kernel='linear', probability=True)
    clf.fit(X_train, y_train)
    
    # Evaluate the classifier
    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred))
    # print(r2score(y_test, y_pred))

    print("Train Score: ",clf.score(X_train, y_train))
    
    
    return clf




### Model saving Function

In [5]:
# Step 4: Save the Model 
def save_model(clf, filename='svm_model.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump(clf, f)
    print(f"Model saved as {filename}")



In [22]:
# Step 5: Classify New Faces with the Trained Model
def classify_face(image_path, clf):
    faces = detect_faces(image_path)  # Detect faces in the image
    embeddings = get_embeddings(face_images=faces, model=vgg16_model)  # Generate embeddings for each face
    
    # Classify each face
    for embedding in embeddings:
        prediction = clf.predict([embedding])  # Predict using the trained model
        print(f"Predicted Label: {prediction[0]}")
        print(f"Predicted Label: {type(prediction[0])}")
        

### Load Your Dataset
Loading my dataset from my_faces/ directory, asking glob to request for every image with the extension .jpg

In [ ]:
# Example: Define paths to your images (small dataset)
image_paths = glob.glob("my_faces/*.jpg")  # Tried using glob to load image from a folder
print(image_paths)

# Collect faces and generate embeddings

all_embeddings = []
all_labels = []
actual_names = ['Tomiwa', 'Samuel', 'Ransome', 'Ifechukwu', 'Seun', 'Christabel' ]
current_label = 0

label_map = {}  # Example label mapping
the_cropped_paths = []
train_images =[]

for image_path in image_paths:
    print(image_path)
    label_name = os.path.basename(image_path).split('_')  # Assuming image names follow format 'person_1_x.jpg'
    print("----------------------------",label_name)
    label_name=label_name[0]+"_"+label_name[1]
    the_cropped_paths.append('cropped/'+label_name)
    # print(label_name.split('.')[0])

    if label_name not in label_map:
        label_map[f'{label_name}'] = current_label
        current_label += 1  # Increment label number
    faces = detect_faces(image_path)
    embeddings = get_embeddings(face_images=faces, model=vgg16_model)
    all_embeddings.extend(embeddings)
    all_labels.extend([label_map[label_name]] * len(embeddings))  # Assign the label for each face

# 


['my_faces\\person_1_1.jpg', 'my_faces\\person_1_2.jpg', 'my_faces\\person_1_3.jpg', 'my_faces\\person_1_4.jpg', 'my_faces\\person_1_5.jpg', 'my_faces\\person_1_6.jpg', 'my_faces\\person_2_1.jpg', 'my_faces\\person_2_10.jpg', 'my_faces\\person_2_11.jpg', 'my_faces\\person_2_12 (2).jpg', 'my_faces\\person_2_12 (3).jpg', 'my_faces\\person_2_12 (4).jpg', 'my_faces\\person_2_12 (5).jpg', 'my_faces\\person_2_12 (6).jpg', 'my_faces\\person_2_12 (7).jpg', 'my_faces\\person_2_12 (8).jpg', 'my_faces\\person_2_12 (9).jpg', 'my_faces\\person_2_12.jpg', 'my_faces\\person_2_13.jpg', 'my_faces\\person_2_14.jpg', 'my_faces\\person_2_15.jpg', 'my_faces\\person_2_16.jpg', 'my_faces\\person_2_17.jpg', 'my_faces\\person_2_19.jpg', 'my_faces\\person_2_2.jpg', 'my_faces\\person_2_20.jpg', 'my_faces\\person_2_3.jpg', 'my_faces\\person_2_4.jpg', 'my_faces\\person_2_5.jpg', 'my_faces\\person_2_6.jpg', 'my_faces\\person_2_7.jpg', 'my_faces\\person_2_8.jpg', 'my_faces\\person_2_9.jpg', 'my_faces\\person_3_1.jpg

5. Train the SVM Classifier
In the next cell, train the SVM classifier using the generated embeddings:

In [19]:
label_map


{'person_1': 0,
 'person_2': 1,
 'person_3': 2,
 'person_4': 3,
 'person_5': 4,
 'person_6': 5}

In [20]:
# Step 3: Train the SVM Classifier
clf = train_svm(all_embeddings, all_labels)


              precision    recall  f1-score   support

           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         2

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

Train Score:  1.0


6. Save the Model (Optional)
If you want to save the trained model for later use, execute this in a new cell:

In [31]:
# Optional: Save the trained model
save_model(clf)


Model saved as svm_model.pkl


7. Test the Classifier on New Images
Finally, use the classifier to classify a new image. In a new cell, call the classify_face function with a test image:

In [34]:
# Step 5: Classify a new image (example)
test_image = 'my_faces/WIN_20250331_10_02_42_Pro.jpg'  # Replace with your test image path
classify_face(test_image, clf)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Label: 1
Predicted Label: <class 'numpy.int64'>


Summary of Notebook Cells
Cell 1: Install dependencies (pip install ...).

Cell 2: Import necessary libraries.

Cell 3: Define helper functions for face detection, embedding generation, SVM training, and saving the model.

Cell 4: Load images and preprocess them (detect faces, generate embeddings).

Cell 5: Train the SVM classifier.

Cell 6: Save the trained model (optional).

Cell 7: Test the classifier on a new image.

# 